In [1]:
import pickle
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')


from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.model_selection import GridSearchCV



In [2]:
train_df = pd.read_csv(r'C:\projects\Python\mlcourse_open\Lesson4\kagel\train_sessions.csv', index_col='session_id')
test_df = pd.read_csv(r'C:\projects\Python\mlcourse_open\Lesson4\kagel\test_sessions.csv', index_col='session_id')

In [3]:
# приведем колонки time1, ..., time10 к временному формату
times = ['time%s' % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# отсортируем данные по времени
train_df = train_df.sort_values(by='time1')

# посмотрим на заголовок обучающей выборки
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


In [292]:
test_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
1,29,2014-10-04 11:19:53,35,2014-10-04 11:19:53,22,2014-10-04 11:19:54,321,2014-10-04 11:19:54,23,2014-10-04 11:19:54,2211,2014-10-04 11:19:54,6730,2014-10-04 11:19:54,21,2014-10-04 11:19:54,44582,2014-10-04 11:20:00,15336,2014-10-04 11:20:00
2,782,2014-07-03 11:00:28,782,2014-07-03 11:00:53,782,2014-07-03 11:00:58,782,2014-07-03 11:01:06,782,2014-07-03 11:01:09,782,2014-07-03 11:01:10,782,2014-07-03 11:01:23,782,2014-07-03 11:01:29,782,2014-07-03 11:01:30,782,2014-07-03 11:01:53
3,55,2014-12-05 15:55:12,55,2014-12-05 15:55:13,55,2014-12-05 15:55:14,55,2014-12-05 15:56:15,55,2014-12-05 15:56:16,55,2014-12-05 15:56:17,55,2014-12-05 15:56:18,55,2014-12-05 15:56:19,1445,2014-12-05 15:56:33,1445,2014-12-05 15:56:36
4,1023,2014-11-04 10:03:19,1022,2014-11-04 10:03:19,50,2014-11-04 10:03:20,222,2014-11-04 10:03:21,202,2014-11-04 10:03:21,3374,2014-11-04 10:03:22,50,2014-11-04 10:03:22,48,2014-11-04 10:03:22,48,2014-11-04 10:03:23,3374,2014-11-04 10:03:23
5,301,2014-05-16 15:05:31,301,2014-05-16 15:05:32,301,2014-05-16 15:05:33,66,2014-05-16 15:05:39,67,2014-05-16 15:05:40,69,2014-05-16 15:05:40,70,2014-05-16 15:05:40,68,2014-05-16 15:05:40,71,2014-05-16 15:05:40,167,2014-05-16 15:05:44


# Целевой признак Target
   

## 1. Подготавливаем данные! Убираем нули.


In [4]:
sites = ['site%s' % i  for i in range(1, 11)]

In [5]:
train_df[sites] = train_df[sites].fillna(0).astype('int')
test_df[sites] = test_df[sites].fillna(0).astype('int')

In [65]:
# загрузим словарик сайтов
with open(r"C:\projects\Python\mlcourse_open\Lesson4\site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)

In [66]:
# датафрейм словарика сайтов
sites_dict_df = pd.DataFrame(list(site_dict.keys()), 
                          index=list(site_dict.values()), 
                          columns=['site'])

In [40]:
sites_dict_df.shape[0]

48371

## Соединяем Обучающиюсяя и тестовую выборку.Для надежности!

In [37]:
y_train = train_df['target']

In [6]:
# объединенная таблица исходных данных
full_df = pd.concat([train_df.drop('target', axis=1), test_df])

# индекс, по которому будем отделять обучающую выборку от тестовой
idx_split = train_df.shape[0]

## Отбираем только сайты для работы c ним.

In [15]:
# табличка с индексами посещенных сайтов в сессии
full_sites = full_df[sites]
full_sites.shape

(336358, 10)

In [15]:
# последовательность с индексами
sites_flatten = full_sites.values.flatten()
sites_flatten[:100]

array([  56,   55,    0,    0,    0,    0,    0,    0,    0,    0,   56,
         55,   56,   55,    0,    0,    0,    0,    0,    0,  946,  946,
        951,  946,  946,  945,  948,  784,  949,  946,  945,  948,  949,
        948,  945,  946,  947,  945,  946,  946,  947,  950,  948,  947,
        950,  952,  946,  951,  946,  947,  952,  947,  953,  946,  947,
        946,  953,  955,  946,  947,  953,  947,  946,  953,  955,  947,
        953,  946,  953, 1033,  946,  947,  954,  953,  946,  954,  946,
        956,  957,  956,  946,  956,  946,  946,  955,  954,  946,  946,
        946,  948,  948,  946,  948,  784,   49,   53,  812,  982,   52,
         52])

In [68]:
# топ-сайты вj dctq ds,jhrt
top_sites = pd.Series(sites_flatten).value_counts().sort_values(ascending=False).head(5)
print(top_sites)
sites_dict_df.loc[top_sites.index]

0      168150
21     160951
23     111862
782    108810
22      75913
dtype: int64


,site
0,NaN
21,www.google.fr
23,www.google.com
782,annotathon.org
22,apis.google.com


## Задание 2: Какие сайты Элис посещает в сети наиболее часто?


    видеохостинги
    социальные сети
    торрент-трекеры
    новостные сайты



In [119]:
Alice_site = train_df[train_df['target'] == 1].drop(times, axis = 1)
# Alice_site = Alice_site.drop(times, axis = 1)
# Alice site

top_alice_site = pd.Series(Alice_site.values.flatten()).value_counts().sort_values(ascending=False).head(5)
top_alice_site

1     2309
77    1382
80    1354
76    1307
29     897
dtype: int64

In [98]:
sites_dict_df.loc[top_alice_site.index]

,site
1,fpdownload2.macromedia.com
77,i1.ytimg.com
80,s.youtube.com
76,www.youtube.com
29,www.facebook.com


## Отбираем Data Frame со времним и поработаем с ним.

In [16]:
time_df = train_df.drop(sites, axis = 1).drop(times, axis = 1)
time_df.head()

,target
session_id,
21669,0
54843,0
77292,0
114021,0
146670,0


In [155]:
time_df['max'] = train_df_time[times].max(axis = 1)
time_df['min'] = train_df_time[times].min(axis = 1)

In [157]:
# вычислим длительность сессии и переведем в секунды
time_df['seconds'] = (time_df['max'] - time_df['min']) / np.timedelta64(1, 's')


In [158]:
time_df.head()

,target,max,min,seconds
session_id,,,,
21669,0,2013-01-12 08:05:57,2013-01-12 08:05:57,0.0
54843,0,2013-01-12 09:07:09,2013-01-12 08:37:23,1786.0
77292,0,2013-01-12 08:50:17,2013-01-12 08:50:13,4.0
114021,0,2013-01-12 08:50:20,2013-01-12 08:50:17,3.0
146670,0,2013-01-12 08:50:22,2013-01-12 08:50:20,2.0


In [163]:
time_df.groupby('target')['seconds'].mean()

target
0    139.282372
1     52.296474
Name: seconds, dtype: float64

In [183]:
time_df_ses40 = time_df[time_df['seconds'] > 40]
time_df_ses40.groupby('target')['seconds'].count()

target
0    107938
1       539
Name: seconds, dtype: int64

## Начинаем строить первую модель


In [17]:
# табличка с индексами посещенных сайтов в сессии
full_sites = full_df[sites]
full_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
54843,56,55,56,55,0,0,0,0,0,0
77292,946,946,951,946,946,945,948,784,949,946
114021,945,948,949,948,945,946,947,945,946,946
146670,947,950,948,947,950,952,946,951,946,947


In [18]:
# последовательность с индексами
sites_flatten = full_sites.values.flatten()
sites_flatten[:100], \
print(len(sites_flatten))

3363580


(array([  56,   55,    0,    0,    0,    0,    0,    0,    0,    0,   56,
          55,   56,   55,    0,    0,    0,    0,    0,    0,  946,  946,
         951,  946,  946,  945,  948,  784,  949,  946,  945,  948,  949,
         948,  945,  946,  947,  945,  946,  946,  947,  950,  948,  947,
         950,  952,  946,  951,  946,  947,  952,  947,  953,  946,  947,
         946,  953,  955,  946,  947,  953,  947,  946,  953,  955,  947,
         953,  946,  953, 1033,  946,  947,  954,  953,  946,  954,  946,
         956,  957,  956,  946,  956,  946,  946,  955,  954,  946,  946,
         946,  948,  948,  946,  948,  784,   49,   53,  812,  982,   52,
          52]), None)

In [19]:
# искомая матрица
full_sites_sparse = csr_matrix(([1] * sites_flatten.shape[0],
                                sites_flatten,
                                range(0, sites_flatten.shape[0] + 10, 10)))[:, 1:]

In [186]:
# Сколько места занимает разреженная матрица в памяти?
print('{0} elements * {1} bytes = {2} bytes'.format(full_sites_sparse.count_nonzero(), 8, 
                                                    full_sites_sparse.count_nonzero() * 8))
# или сразу вот так:
print('sparse_matrix_size = {0} bytes'.format(full_sites_sparse.data.nbytes))

1866898 elements * 8 bytes = 14935184 bytes
sparse_matrix_size = 7467592 bytes


## Тестовый вариант разреженной таблицы

In [217]:
# данные, создаем список единичек, длина которого равна количеству элементов в исходной таблице (9)
# просуммировав количество единичек в ячейке получим частоту, 
# сколько было посещений определенного сайта за сессию
data = [1] * 9

# для этого надо правильно распределить единички по ячейкам
# индексы - номера сайтов, по ним будут суммироваться единички за сессии, т.е. колонки новой матрицы
indices = [1, 0, 0, 1, 3, 1, 2, 3, 4]

# индексы разбиения на строки (сессии)
# например, строка 0 это элементы между индексами [0; 3) - крайнее правое значение не включается
# строка 1 это элементы между индексами [3; 6) 
# строка 2 это элементы между индексами [6; 9) 
indptr = [0, 3, 6, 9]

# объединим эти три переменных в кортеж и сформируем матрицу
# чтобы вывести на экран преобразуем в обычную "плотную" матрицу
my_test_matr = csr_matrix((data, indices, indptr)).todense()
my_test_matr

matrix([[2, 1, 0, 0, 0],
        [0, 2, 0, 1, 0],
        [0, 0, 1, 1, 1]])

In [219]:
my_test_matr.data.nbytes

60

## Строим первую модель!

In [44]:
def get_auc_lr_valid(X, y, C=1.0, seed=17, ratio = 0.9):
    # разделим выборку на обучающую и валидационную
    idx = int(round(X.shape[0] * ratio))
    # обучение классификатора
    lr = LogisticRegression(C=C, random_state=seed, n_jobs=-1).fit(X[:idx, :], y[:idx])
    # прогноз для валидационной выборки
    y_pred = lr.predict_proba(X[idx:, :])[:, 1]
    # считаем качество
    score = roc_auc_score(y[idx:], y_pred)
    
    return score



In [223]:
%%time
# выделим из объединенной выборки только обучающую (для которой есть ответы)
X_train = full_sites_sparse[:idx_split, :]

# считаем метрику на валидационной выборке
print(get_auc_lr_valid(X_train, y_train))

C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


0.919524105836
Wall time: 6.12 s


In [240]:
# функция для записи прогнозов в файл
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

## Обучаем логистическую модель на всей выборке!

In [243]:
X_train_sparse = full_sites_sparse[:idx_split]
X_test_sparse = full_sites_sparse[idx_split:]



In [244]:
X_train_sparse.shape, y_train.shape

((253561, 48371), (253561,))

In [245]:
%%time
logit = LogisticRegression(n_jobs=-1, random_state=17)
logit.fit(X_train_sparse, y_train)



C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


Wall time: 5.86 s


In [246]:
test_pred = logit.predict_proba(X_test_sparse)[:, 1]

## Формируем правельный документ дя отправки

In [248]:
Send  = pd.Series(test_pred, index=range(1, test_pred.shape[0] + 1),
         name='target').to_csv('benchmark1.csv', header=True, index_label='session_id')



## Создаем новые дата фреймы! Улучшаем Модель

In [7]:
new_feat_train = pd.DataFrame(index=train_df.index)
new_feat_test = pd.DataFrame(index=test_df.index)

In [10]:
new_feat_train['year_month'] = train_df['time1'].apply(lambda ts: 100 * ts.year + ts.month)
new_feat_test['year_month'] = test_df['time1'].apply(lambda ts: 100 * ts.year + ts.month)

## Отмасштабируем признал для логистической регрессии(Чтобы вес не скакал)

In [11]:
scaler = StandardScaler()
scaler.fit(new_feat_train['year_month'].values.reshape(-1, 1))

new_feat_train['year_month_scaled'] = scaler.transform(new_feat_train['year_month'].values.reshape(-1, 1))
new_feat_test['year_month_scaled'] = scaler.transform(new_feat_test['year_month'].values.reshape(-1, 1))



C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [12]:
new_feat_train.head()

,year_month,year_month_scaled
session_id,,
1,201402,0.634518
2,201402,0.634518
3,201312,-1.485314
4,201403,0.658072
5,201402,0.634518


In [269]:
X_train_sparse_new = csr_matrix(hstack([X_train_sparse, 
                             new_feat_train['year_month_scaled'].values.reshape(-1, 1)]))

In [272]:
%%time
Log_reg_2 = LogisticRegression(C = 1.0, random_state=17).fit(X_train_sparse_new, y_train)

Wall time: 5.27 s


In [276]:
get_auc_lr_valid(X_train_sparse_new, y_train)

C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


0.91968925566038195

In [38]:
# датафрейм для новых признаков
full_new_feat = pd.DataFrame(index=full_df.index)

# добавим признак start_month
full_new_feat['start_month'] = full_df['time1'].apply(lambda ts: 100 * ts.year + ts.month)

In [39]:
full_new_feat['start_month'].head()

session_id
21669     201301
54843     201301
77292     201301
114021    201301
146670    201301
Name: start_month, dtype: int64

## Я напомни себе как сделать категориальный признак из числового

### Ручной способ через свою функцию

In [22]:
full_new_feat['start_month'] = full_new_feat['start_month'].apply(lambda ts: 0 if ts > 201301
                                                                                else 1)

In [26]:
full_new_feat['start_month'].value_counts()

0    335995
1       363
Name: start_month, dtype: int64

## Есть специальный метод который разбивает Data на признаки

In [40]:
start_session_categor = pd.get_dummies(full_new_feat['start_month'])

In [41]:
start_session_categor.head()

,201301,201302,201303,201304,201305,201306,201307,201308,201309,201310,...,201403,201404,201405,201406,201407,201408,201409,201410,201411,201412
session_id,,,,,,,,,,,,,,,,,,,,,
21669,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
54843,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
77292,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
114021,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
146670,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
train_df = pd.concat([train_df, pd.get_dummies(train_df['Pclass'], 
                                               prefix="PClass"),
                      pd.get_dummies(train_df['Sex'], prefix="Sex"),
                      pd.get_dummies(train_df['SibSp'], prefix="SibSp"),
                      pd.get_dummies(train_df['Parch'], prefix="Parch"),
                     pd.get_dummies(train_df['Embarked'], prefix="Embarked")],
                     axis=1)

In [ ]:
pd.get_dummies?

In [42]:
pd.concat?

## Добавляем признак без масштабирования! Веса нарушатся! Качесто ухудшается!

In [54]:
# добавим новый признак в разреженную матрицу
tmp = full_new_feat[['start_month']].as_matrix()
X_train = csr_matrix(hstack([full_sites_sparse[:idx_split,:], tmp[:idx_split,:]]))

# считаем метрику на валидационной выборке
print(get_auc_lr_valid(X_train, y_train))

C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


0.750835486018


## Масштабируем признак

In [55]:
# добавим новый стандартизированный признак в разреженную матрицу
tmp = StandardScaler().fit_transform(full_new_feat[['start_month']])
X_train = csr_matrix(hstack([full_sites_sparse[:idx_split,:], tmp[:idx_split,:]]))

# считаем метрику на валидационной выборке
print(get_auc_lr_valid(X_train, y_train))



C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


0.919698615157


In [202]:
full_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
54843,56,55,56,55,0,0,0,0,0,0
77292,946,946,951,946,946,945,948,784,949,946
114021,945,948,949,948,945,946,947,945,946,946
146670,947,950,948,947,950,952,946,951,946,947


## Стоит задача найти уникальное количeство сайтов в сессии

Проблема в том что при объединение 2-х выборок.Тренировочной и Тестовов совпадаю сессии.Поэтом к полнымм данным не удается применить сетод Юник.Так что посчитаю только по тренеровочной выборке.

In [22]:
train_df_sites = train_df[sites]

In [145]:
full_sites.loc[21669]

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
21669,21,42279,76,21,229,21,234,42279,76,42279


In [236]:
x = pd.unique(train_df_sites.loc[21669].values)


array([56, 55,  0], dtype=int64)

## Моя попытка сделать руками, в целом рабочая но есть специальный метод!

In [23]:
train_df_sites['uniq_count_site'] = train_df_sites.apply(lambda s:pd.unique(s).shape[0], axis=1)
train_df_sites.head()

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,uniq_count_site
session_id,,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0,3
54843,56,55,56,55,0,0,0,0,0,0,3
77292,946,946,951,946,946,945,948,784,949,946,6
114021,945,948,949,948,945,946,947,945,946,946,5
146670,947,950,948,947,950,952,946,951,946,947,6


In [179]:
ts = full_sites.loc[21669].values
ts

array([[   56,    55,     0,     0,     0,     0,     0,     0,     0,
            0],
       [   21, 42279,    76,    21,   229,    21,   234, 42279,    76,
        42279]])

In [240]:
list1 = [1,2,1,3,1]
pd.unique(list1)

array([1, 2, 3], dtype=int64)

In [260]:
df = pd.DataFrame({'A': [1, 2, 3,], 'B': [0, 1, 3]})
df.nunique(axis=1, dropna=False)


0    2
1    2
2    1
dtype: int64

## Здесь я хочу собрать разряженную таблиц на тренировачную выборку!
    

### Добавляю признак начало сессии,уникальные сайты, и время сессии.

### Делаю разрежанную таблицу с сайтами

In [24]:
train_df_sites['uniq_count_site'] = train_df_sites.nunique(axis = 1, dropna=True)


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [25]:
# Получаю список со всеми номерами сайта
sites_train_flatt = train_df_sites.values.flatten()
sites_train_flatt.shape[0]

2789171

In [26]:
# искомая матрица( Здесь есть боьшой вопрос нужно ли разрежать уникальные сайты? и нужно ли масштабировать этот признак?)
train_sites_sparse = csr_matrix(([1] * sites_train_flatt.shape[0],
                                sites_train_flatt,
                                range(0, sites_train_flatt.shape[0] + 11, 11)))[:, 1:]


In [331]:
train_sites_sparse.shape

(253561, 41601)

### Сделаю признак Стартовый месяц

In [270]:
# датафрейм для новых признаков
train_month_feat = pd.DataFrame(index=train_df.index)

# добавим признак start_month
train_month_feat['start_month'] = train_df['time1'].apply(lambda ts: 100 * ts.year + ts.month)
train_month_feat.head()

,start_month
session_id,
21669,201301
54843,201301
77292,201301
114021,201301
146670,201301


In [332]:
# Это обученный масштабатор с начальном сессии
tmp = StandardScaler().fit_transform(train_month_feat[['start_month']])
tmp.shape

(253561, 1)

## Объеденяю Таблицу с сайтами+Уникальными сайтам, и Стартовый месяц(отмасштабированный)

In [333]:
# добавим новый стандартизированный признак в разреженную матрицу
X_train_site_month = csr_matrix(hstack([train_sites_sparse, tmp]))



In [361]:
X_train_site_month.shape

(253561, 41602)

### Алгоритм регресси на тренировачной выборке!

In [354]:
# обучение классификатора
log_reg_train = LogisticRegression(C=1.0, random_state=17, n_jobs=-1).fit(X_train_site_month, y_train)
    
# прогноз для валидационной выборки
y_pred = log_reg_train.predict_proba(X_train_site_month)[:, 1]

# считаем качество
score = roc_auc_score(y_train, y_pred)

C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


In [356]:
print(score)

0.980248244501


### Алгоритм регресси на тестовой выборке

In [290]:
test_df.head(2)

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
1,29,2014-10-04 11:19:53,35,2014-10-04 11:19:53,22,2014-10-04 11:19:54,321,2014-10-04 11:19:54,23,2014-10-04 11:19:54,2211,2014-10-04 11:19:54,6730,2014-10-04 11:19:54,21,2014-10-04 11:19:54,44582,2014-10-04 11:20:00,15336,2014-10-04 11:20:00
2,782,2014-07-03 11:00:28,782,2014-07-03 11:00:53,782,2014-07-03 11:00:58,782,2014-07-03 11:01:06,782,2014-07-03 11:01:09,782,2014-07-03 11:01:10,782,2014-07-03 11:01:23,782,2014-07-03 11:01:29,782,2014-07-03 11:01:30,782,2014-07-03 11:01:53


### Делаю тестовую выборку пригодной для прогноза! Тоесть разрежаю ее и добавляю теже признаки

In [374]:
test_df_sites = test_df[sites]
test_df_sites.shape

(82797, 10)

In [376]:
test_df_sites['unique_site'] = test_df_sites.nunique(axis=1)


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [377]:
test_df_sites.shape

(82797, 11)

In [378]:
# Получаю список со всеми номерами сайта
sites_test_flatt = test_df_sites.values.flatten()
sites_test_flatt.shape[0]

910767

In [379]:
# искомая матрица( Здесь есть боьшой вопрос нужно ли разрежать уникальные сайты? и нужно ли масштабировать этот признак?)
test_sites_sparse = csr_matrix(([1] * sites_test_flatt.shape[0],
                                sites_test_flatt,
                                range(0, sites_test_flatt.shape[0] + 11, 11)))[:, 1:]

In [380]:
test_sites_sparse.shape

(82797, 48371)

In [343]:
# датафрейм для новых признаков
test_month_feat = pd.DataFrame(index=test_df.index)

# добавим признак start_month
test_month_feat['start_month'] = test_df['time1'].apply(lambda ts: 100 * ts.year + ts.month)


In [351]:
# Это обученный масштабатор с начальном сессии
testmp = StandardScaler().fit_transform(test_month_feat[['start_month']])
testmp.shape

(82797, 1)

### Подготовленная тестовая таблица для построения прогноза!

In [371]:
X_test_sparce = csr_matrix(hstack([test_sites_sparse, testmp]))
X_test_sparce.shape

(82797, 48372)

Вообщем по всей видемости нужно выборки будет Объеденять так как отсутсвуют каке то сайты а обучениитоесть обучится надо будет в том числе и а тестовой выборке! Это я сделаю Завтра! Я молодец

### Сейчас стоит задача обучиться на всей выборке full site+uniq+month
    

### 1. Добавляю признак уникальные сайты для все выборки.

In [16]:
full_sites = full_df[sites]


In [28]:
full_sites['unique_sites'] = full_df[sites].nunique(axis = 1)
full_sites.head()

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,unique_sites
session_id,,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0,3
54843,56,55,56,55,0,0,0,0,0,0,3
77292,946,946,951,946,946,945,948,784,949,946,6
114021,945,948,949,948,945,946,947,945,946,946,5
146670,947,950,948,947,950,952,946,951,946,947,6


#### Делаю признак масштабируемым и добавляю к разряженной табилце

In [29]:
## Сделаю признак unique_site масштабируемым и прикручу его к таблице
scaller_unique_sites = StandardScaler().fit_transform(full_sites[['unique_sites']])
print(scaller_unique_sites)

[[-1.07415153]
 [-1.07415153]
 [ 0.14600956]
 ..., 
 [ 1.77289101]
 [ 0.55272992]
 [-1.48087189]]


In [30]:
full_sites_scaller_unique = full_df[sites]

In [32]:
full_sites_scaller_unique['unique_sites'] = scaller_unique_sites

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [77]:
full_sites_scaller_unique.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,unique_sites
session_id,,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0,-1.074152
54843,56,55,56,55,0,0,0,0,0,0,-1.074152
77292,946,946,951,946,946,945,948,784,949,946,0.146010
114021,945,948,949,948,945,946,947,945,946,946,-0.260711
146670,947,950,948,947,950,952,946,951,946,947,0.146010


### 2. Делаем разреженную таблицу full_sites и full_sites + uniqe!

In [17]:
# Делаем список значений для разреженной таблицы! Только sites

full_sites_flatt = full_sites.values.flatten()
full_sites_flatt.shape

(3363580,)

In [18]:
full_sites_pars = csr_matrix(([1] * full_sites_flatt.shape[0],
                              full_sites_flatt,
                             range(0, full_sites_flatt.shape[0] + 10, 10)))[:, 1:]
full_sites_pars.shape

(336358, 48371)

### Разреженная таблица + уникальные сайты

In [36]:
# Делаем список значений для разреженной таблицы + разрежаем уникальные сайты

full_sites_unique_flatt = full_sites.values.flatten()
full_sites_unique_flatt


array([  56,   55,    0, ..., 1098, 1098,    2], dtype=int64)

In [37]:
full_sites_unique_pars = csr_matrix(([1] * full_sites_unique_flatt.shape[0],
                                     full_sites_unique_flatt,
                                    range (0, full_sites_unique_flatt.shape[0] + 11, 11)))[:, 1:]
full_sites_unique_pars.shape

(336358, 48371)

### 3. Делаем признак старт Month и Масштабируем его.

In [32]:
# Передаем нужный индекс
time_start_month = pd.DataFrame(index = full_df.index)

In [33]:
time_start_month['start_month'] = full_df['time1'].apply(lambda ts: 100 * ts.year + ts.month)
time_start_month.head()

,start_month
session_id,
21669,201301
54843,201301
77292,201301
114021,201301
146670,201301


In [34]:
# Масштабируем признак чтобы числа были не болшими
scaller = StandardScaler()
scaller_start_month = scaller.fit_transform(time_start_month[['start_month']])
scaller_start_month.shape

(336358, 1)

### 4. Собираем все в 1 разряженную таблицу по кторой и будем обучатся! Site+unique+startmonth

In [12]:
# Это таблица с разряженными никальными сайтами
X_full_pars = csr_matrix(hstack([full_sites_unique_pars, scaller_start_month]))
X_full_pars.shape

NameError: name 'full_sites_unique_pars' is not defined

In [43]:
# Это таблица с разряженными  сайтами + уникальные+начало месяца
X_sites_uniq_month_pars = csr_matrix(hstack([full_sites_pars, scaller_unique_sites, scaller_start_month]))
X_sites_uniq_month_pars.shape

(336358, 48373)

### 5. Проводим обучение на тренировочной выборке с ответами!

In [59]:
# индекс, по которому будем отделять обучающую выборку от тестовой
idx_split = train_df.shape[0]

In [49]:
logreg_3feat = LogisticRegression(C=1.0, random_state=17).fit(X_full_pars[:idx_split], y_train)

In [50]:
# прогноз для валидационной выборки
y_pred_3feat = logreg_3feat.predict_proba(X_full_pars[:idx_split])[:, 1]

# считаем качество
score = roc_auc_score(y_train, y_pred_3feat)
score

0.98016790191075165

In [52]:
# Идет переобучние! Нкжно брать 80% от трен выборки

### 5.1 Обучаем на разреженной по сайтам + отмасштабируемой по началу и уникальным сайтам

In [44]:
# Обучусь на 80%
idx80 = round(X_sites_uniq_month_pars[:idx_split].shape[0] * 0.8)
print(idx80)

202849


In [45]:
X_train80 = X_sites_uniq_month_pars[:idx_split]

In [46]:
logreg2_3feat = LogisticRegression(C = 1.0, random_state=17, n_jobs= -1).fit(X_sites_uniq_month_pars[:idx80], y_train[:idx80])

C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


In [47]:
y2_pred3feat = logreg2_3feat.predict_proba(X_train80[idx80:])[:, 1]
score = roc_auc_score(y_train[idx80:], y2_pred3feat)
score

0.91650979755124928

In [66]:
Send  = pd.Series(y_pred_3feat_test, index=range(1, y_pred_3feat_test.shape[0] + 1),
         name='target').to_csv('benchmark5.csv', header=True, index_label='session_id')

### 6.2 Второе предсказание

In [93]:
y2_pred_3feat_test = logreg2_3feat.predict_proba(X_sites_uniq_month_pars[idx_split :])[:, 1]

In [94]:
Send  = pd.Series(y_pred_3feat_test, index=range(1, y_pred_3feat_test.shape[0] + 1),
         name='target').to_csv('benchmark4.csv', header=True, index_label='session_id')

### 7. Нужно добавить 2 новых признака!Start hour и Morning/ до 11

In [22]:
full_df['start_hour'] = full_df['time1'].apply(lambda ts: ts.hour)

In [23]:
full_df['morning'] = full_df['start_hour'].apply(lambda ts: 1 if ts < 11
                                                               else 0)

In [24]:
hour_df = pd.DataFrame(index = full_df.index)
hour_df['hour'] = full_df['start_hour']


In [25]:
morning_df = pd.DataFrame(index = full_df.index)
morning_df['morning'] = full_df['morning']


## 8. Сейчас я хочу проверить нужно ли мастштабировать часы или нет(просто сайты)

### 8.1 На потребуется: Распарсенная таблица всех сайтов+ скалированный Hour  и Обычный(от 0 до 24)

In [99]:
full_sites_pars.shape

(336358, 48371)

In [83]:
hour.shape

(336358, 1)

###  Метод сделать скалирование

In [26]:
scaller_hour = StandardScaler().fit_transform(hour_df)
scaller_hour;

### 8.2 Соеденяем 2 Разреженные матрицы

In [117]:
X_sites_hour = csr_matrix(hstack([full_sites_pars, hour_df]))

In [118]:
X_sites_scallhour = csr_matrix(hstack([full_sites_pars, scaller_hour]))

### 8.3 Обучаем модель и делаем 2 прогноза

In [95]:
def get_auc_lr_valid(X, y, C=1.0, seed=17, ratio = 0.9):
    # разделим выборку на обучающую и валидационную
    idx = int(round(X.shape[0] * ratio))
    # обучение классификатора
    lr = LogisticRegression(C=C, random_state=seed, n_jobs=-1).fit(X[:idx, :], y[:idx])
    # прогноз для валидационной выборки
    y_pred = lr.predict_proba(X[idx:, :])[:, 1]
    # считаем качество
    score = roc_auc_score(y[idx:], y_pred)
    
    return score

In [93]:
# индекс, по которому будем отделять обучающую выборку от тестовой
idx_split = train_df.shape[0]

In [119]:
# Модель с чистыми hour
get_auc_lr_valid(X_sites_hour[:idx_split], y_train[:idx_split])

C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


0.95607656249245199

In [120]:
# Модель с скаллерным hour
get_auc_lr_valid(X_sites_scallhour[:idx_split], y_train[:idx_split])


C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


0.9565881646450155

### 8.4 Теперь добавляю признак месяц к сайтам и чистым часам(Стало лучше)

In [123]:
X_train_s_h_m = csr_matrix(hstack([full_sites_pars, scaller_hour, scaller_start_month]))

In [124]:
# Модель с cайты+hour+скалиррованый месяц
get_auc_lr_valid(X_train_s_h_m[:idx_split], y_train[:idx_split])

C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


0.95792430824263686

### 8.5 Добавляю бинарный признак утро!(Он улучшил показатели модели)

In [35]:
X_train_s_h_month_morning = csr_matrix(hstack([full_sites_pars, scaller_hour, scaller_start_month, morning_df]))

In [144]:
# Модель с cайты+hour+скалиррованый месяц+утро
get_auc_lr_valid(X_train_s_h_month_morning[:idx_split], y_train[:idx_split])

C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


0.95851501323613952

In [38]:
Log_reg_best = LogisticRegression(random_state=17).fit(X_train_s_h_month_morning[:idx_split], y_train[:idx_split])

In [39]:
# прогноз для валидационной выборки
y_pred = Log_reg_best.predict_proba(X_train_s_h_month_morning[:idx_split])[:, 1]
# считаем качество
score = roc_auc_score(y_train[:idx_split], y_pred)
score

0.98820307927558837

In [41]:
y_test_h = Log_reg_best.predict_proba(X_train_s_h_month_morning[idx_split:])[:, 1]

In [43]:
Send  = pd.Series(y_test_h, index=range(1, y_test_h.shape[0] + 1),
         name='target').to_csv('benchmark5.csv', header=True, index_label='session_id')

## 9.0 Пробую использовать регуляризацию для улучшения качества


In [160]:
GridSearchCV?

In [7]:
%%time
from sklearn.model_selection import GridSearchCV

Wall time: 0 ns


In [8]:
param_grid_logit = {'logisticregression__C': np.logspace(-3, 1, 10)}

In [40]:
grid_logit = GridSearchCV(Log_reg_best, param_grid_logit, cv=3)

In [ ]:
%%time
from sklearn.model_selection import GridSearchCV

param_grid_logit = {'logisticregression__C': np.logspace(-5, 0, 6)}
grid_logit = GridSearchCV(text_pipe_logit, param_grid_logit, cv=3, n_jobs=-1)

grid_logit.fit(text_train, y_train)

CPU times: user 26.7 s, sys: 1.33 s, total: 28.1 s
Wall time: 1min 16s

Лучшее значение C и соответствующее качество на кросс-валидации:
In [23]:

grid_logit.best_params_, grid_logit.best_score_

Out[23]:

({'logisticregression__C': 0.10000000000000001}, 0.88527999999999996)

In [26]:

plot_grid_scores(grid_logit, 'logisticregression__C')



### 9.1 


In [44]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

In [45]:
c_values = np.logspace(-3, 1, 10)

In [46]:
logit_searcher = LogisticRegressionCV(Cs=c_values, cv=skf, verbose=1, n_jobs=-1)

logit_searcher.fit(X_train_s_h_month_morning[:idx_split], y_train[:idx_split])

[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.3min finished


LogisticRegressionCV(Cs=array([  1.00000e-03,   2.78256e-03,   7.74264e-03,   2.15443e-02,
         5.99484e-02,   1.66810e-01,   4.64159e-01,   1.29155e+00,
         3.59381e+00,   1.00000e+01]),
           class_weight=None,
           cv=StratifiedKFold(n_splits=5, random_state=17, shuffle=True),
           dual=False, fit_intercept=True, intercept_scaling=1.0,
           max_iter=100, multi_class='ovr', n_jobs=-1, penalty='l2',
           random_state=None, refit=True, scoring=None, solver='lbfgs',
           tol=0.0001, verbose=1)

In [48]:
y_pred_regul = logit_searcher.predict_proba(X_train_s_h_month_morning[:idx_split])[:, 1]
# считаем качество
score = roc_auc_score(y_train[:idx_split], y_pred_regul)
score

0.99076358395166919

In [52]:
y_send_pdict = logit_searcher.predict_proba(X_train_s_h_month_morning[idx_split:])[:, 1]

In [54]:
Send  = pd.Series(y_send_pdict, index=range(1, y_send_pdict.shape[0] + 1),
         name='target').to_csv('benchmark6regul.csv', header=True, index_label='session_id')